In [ ]:
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Loading Datasets and divide into Train and Test 

### Two ways 
    1. with load_data()
    2. by downloading .npz   downlaod-link = https://www.kaggle.com/datasets/vikramtiwari/mnist-numpy()



In [ ]:
# loading data with keras builtin method load_data() - https://keras.io/api/datasets/mnist/#load_data-function (check for reference)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# here (10000,) doesnt have pixel valuelike 28,28 in train dataset because it is a 1-d list of labels of 0-9 digits 
print(f"X_train shape {x_train.shape}")
print(f"y_train shape {y_train.shape}")
print(f"x_test shape {x_test.shape}")
print(f"y_test shape {y_test.shape} ")

print('\n')

# printing the dimensions of each case
print(f"dimension of x_train is : {np.ndim(x_train)}-D")
print(f"dimension of y_train is : {np.ndim(y_train)}-D")
print(f"dimension of x_test  is : {np.ndim(x_test)}-D")
print(f"dimension of y_test  is : {np.ndim(y_test)}-D")


In [ ]:
# laod dataset through dwnloading .npz file of mnist dataset
dataset_path = 'C:/Users/DELL/Downloads/mnist.npz'
data = np.load(dataset_path)       # np.load loads  pickel/.npz/.npy file format

# The .npz file usually contains multiple arrays. You need to know the keys associated with each array.
# You can print all available keys to know what data is inside the .npz file
print(f"all keys of datasets are : {data.files}")

# Assuming the keys are ['x_test', 'x_train', 'y_train', 'y_test']
x_train = data['x_train']
y_train = data['y_train']
x_test = data['x_test']
y_test = data['y_test']


print(f"X_train shape {x_train.shape}")
print(f"X_train shape {y_train.shape}")
print(f"X_train shape {x_test.shape}")
print(f"X_train shape {y_test.shape}")

data.close

# Visualize the Dataset image

### Single image visualization

In [ ]:
# function to visualize an image inside the dataset

def printing_single_image(npz_file_path , image_index):
    dataset_path = npz_file_path
    all_data = np.load(dataset_path) #loading the dataset

    image_index = image_index
    image = all_data['x_train'][image_index]
    label = all_data['y_train'][image_index]

    print(f"Dimension of this image is {np.ndim(image)}-D")             #output the dimension of the image
    print(f"raw matrics {image}")    #print the raw matrics for output image

    plt.imshow(image)               # takes numpy array and show image into visual format
    plt.title(f'Label: {label}')    
    # plt.axis('off')  # Turn off axis numbers and ticks
    plt.show()


npz_file_path = 'C:/Users/DELL/Downloads/mnist.npz'     
image_index = 25
printing_single_image(npz_file_path,image_index)

### Multiple image visualization

In [ ]:
def plot_images(images, labels, nrows, ncols):
    """Plot a grid of images with their labels."""
    fig, axes = plt.subplots(nrows, ncols)
    for i, ax in enumerate(axes.flat):
        if i < len(images):
            ax.imshow(images[i])
            ax.set_title(f"Label: {labels[i]}")
            ax.axis('off')  # Turn off axis numbers and ticks
    plt.show()

# Show 25 images from the dataset
npz_file_path = 'C:/Users/DELL/Downloads/mnist.npz'    
all_data = np.load(npz_file_path) #loading the dataset 

x_train = all_data['x_train']
y_train = all_data['y_train']
plot_images(x_train, y_train, 3,3)


In [ ]:
npz_file_path = 'C:/Users/DELL/Downloads/mnist.npz'    
all_data = np.load(npz_file_path) #loading the dataset 

x_train = all_data['x_train']
y_train = all_data['y_train']

fig , axes = plt.subplots(3,3)
# print(fig)        #give the size of plotting surface
# print(axes)       
ax = axes[1,1]
print(ax)
ax.imshow(x_train[0])
# print(len(x_train))
# print(x_train.shape)
# print(x_train)